In [1]:
import tweepy
from datetime import datetime, timedelta
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os

def lambda_handler(event, context):
    
    # Define database credentials and path
    dbuname = os.environ['DBUNAME']
    dbpwd = os.environ['DBPWD']
    host = os.environ['HOST']
    port = os.environ['PORT']
    dbname = os.environ['DBNAME']
    bearer_token = os.environ['BEARER_TOKEN']
    
    # Define API client function
    client = tweepy.Client(bearer_token=bearer_token)
    
    # Define function for fetching and loading the data
    def fetch_load_tweetcount(label, ht1, ht2, start, end, method):
    
        # Define query, time period and table_name
        query = '('+ ht1 + ' OR ' + ht2 + ') -is:retweet'
        start = start
        end = end
        table_name = 'table_tw_'+label
                
        while True:    
            #fetch data
            try:
                counts = client.get_recent_tweets_count(query=query,start_time = start, end_time = end , granularity='day')
                tweets_df = pd.DataFrame()

                for count in counts.data:
                    tweets_df = tweets_df.append(pd.DataFrame([{'date': count['end'], 
                                                'tweet_count': count['tweet_count']}]))

                tweets_df['date'] = pd.to_datetime(tweets_df['date'])
                tweets_df['label'] = label
                tweets_df = tweets_df.reset_index(drop=True)
            except Exception as e:
                print(e)
                print("tweet count failed: " + label)
                break
        
            #load data
            if_ex_val = method
        
            try:
                conn_string = 'postgresql://'+dbuname+':'+dbpwd+'@'+host+':'+port+'/'+dbname
                engine = create_engine(conn_string)
                tweets_df.to_sql(table_name, conn_string, if_exists = if_ex_val, index=False) 
            except Exception as e:
                print(e)
                print("Data load failed: " + table_name)     
                break
            break
        
    # define hashtags    
    hashtags = {'bitcoin': ['#bitcoin','#btc'],'ethereum': ['#eth', '#ethereum'], 'binance': ['#binance','#bnb'],'ripple': ['#ripple','#xrp'],
           'terra':['#terra','#luna'], 'cardano': ['#cardano','#ada'], 'solana': ['#sol','#solana'], 'avalanche': ['#avalanche','#avax'],
           'polkadot':['#polkadot','#dot'],'dogecoin':['#dogecoin', '#doge'], 'msci_world': ['#msciworld', '#msci'],'euro_stoxx': ['#EXW1.DE','#eurostoxx'],
            'smi':['#CSSMI.SW', '#smi'], 'nasdaq': ['#nasdaq', '#EXXT.DE'], 'gold':['#gold','#GC=F'], 'silver':['#silver','#SI=F']}   
            
    # Daily load of new data

    # get datetime of yesterday and ereyesterday at 00:00:00
    yesterday = datetime.strftime(datetime.today() - timedelta(1), "%Y, %m, %d")
    ereyesterday = datetime.strftime(datetime.today() - timedelta(2), "%Y, %m, %d")

    # set timeframe
    end = datetime.strptime(yesterday, "%Y, %m, %d")
    start = datetime.strptime(ereyesterday, "%Y, %m, %d")
 
    # set method of function df.to_sql()
    method = 'append'

    # execute function
    for label in hashtags:
        ht1 = hashtags[label][0]
        ht2 = hashtags[label][1]
        fetch_load_tweetcount(label, ht1, ht2, start, end, method)
        